In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [2]:
tf.test.is_gpu_available()

True

In [3]:
import tensorflow as tf
#Dense
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        self.units = units
        super(MyDense, self).__init__(**kwargs)

    #build方法一般定义Layer需要被训练的参数。    
    def build(self, input_shape): 
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='w')
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='b')
        super(MyDense,self).build(input_shape) # 相当于设置self.built = True

    #call方法一般定义正向传播运算逻辑，__call__方法调用了它。    
    def call(self, inputs): 
        return tf.matmul(inputs, self.w) + self.b

    #如果要让自定义的Layer通过Functional API 组合成模型时可以序列化，需要自定义get_config方法。
    def get_config(self):  
        config = super(MyDense, self).get_config()
        config.update({'units': self.units})
        return config

In [9]:
from sklearn import datasets
iris = datasets.load_iris()
data = iris.data
labels = iris.target
# from sklearn.preprocessing import MinMaxScaler
# data=MinMaxScaler().fit_transform(data)

In [10]:
data[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [11]:
labels#(150,3)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [4]:
#网络   函数式构建的网络
inputs = tf.keras.Input(shape=(4,))  
x = MyDense(units=16)(inputs) 
x = tf.nn.tanh(x) 
x = MyDense(units=3)(x) #0,1,2
# x= tf.keras.layers.Dense(16)(x)
predictions = tf.nn.softmax(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)


In [5]:
#shuffle:

In [12]:
import numpy as np

In [13]:
data = np.concatenate((data,labels.reshape(150,1)),axis=-1)
np.random.shuffle(data)

In [14]:
labels = data[:,-1]
data = data[:,:4]

In [12]:
#labels  ==[1,0,0]

In [15]:
#优化器 Adam
#损失函数 交叉熵损失函数
#评估函数 #acc


model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

#keras
model.fit(data, labels, batch_size=32, epochs=100,shuffle=True)

Train on 150 samples
Epoch 1/100
150/150 [==============================] - 1s 7ms/sample - loss: 1.0939 - sparse_categorical_accuracy: 0.5333
Epoch 2/100
150/150 [==============================] - 0s 153us/sample - loss: 1.0906 - sparse_categorical_accuracy: 0.6667
Epoch 3/100
150/150 [==============================] - 0s 146us/sample - loss: 1.0873 - sparse_categorical_accuracy: 0.6667
Epoch 4/100
150/150 [==============================] - 0s 146us/sample - loss: 1.0843 - sparse_categorical_accuracy: 0.6533
Epoch 5/100
150/150 [==============================] - 0s 140us/sample - loss: 1.0810 - sparse_categorical_accuracy: 0.6267
Epoch 6/100
150/150 [==============================] - 0s 133us/sample - loss: 1.0776 - sparse_categorical_accuracy: 0.6267
Epoch 7/100
150/150 [==============================] - 0s 133us/sample - loss: 1.0737 - sparse_categorical_accuracy: 0.6467
Epoch 8/100
150/150 [==============================] - 0s 126us/sample - loss: 1.0695 - sparse_categorical_accura

In [14]:
# _custom_objects = {
#     "Mylayer" :  Line,
# }

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
my_dense (MyDense)           (None, 16)                80        
_________________________________________________________________
tf_op_layer_Tanh (TensorFlow [(None, 16)]              0         
_________________________________________________________________
my_dense_1 (MyDense)         (None, 3)                 51        
_________________________________________________________________
tf_op_layer_Softmax (TensorF [(None, 3)]               0         
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.save('keras_model_tf_version.h5')

In [19]:
_custom_objects = {
    "MyDense" :  MyDense,
    
}

In [20]:
new_model = tf.keras.models.load_model("keras_model_tf_version.h5",custom_objects=_custom_objects)

In [21]:
y_pred = new_model.predict(data)

In [22]:
np.argmax(y_pred,axis=1)

array([2, 1, 0, 0, 0, 2, 1, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 0,
       0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 2, 1, 2, 2, 2, 0, 1, 1, 1, 1, 2, 1,
       2, 2, 1, 0, 2, 0, 2, 1, 1, 1, 2, 2, 1, 1, 0, 0, 1, 0, 1, 1, 2, 1,
       1, 2, 2, 0, 1, 1, 0, 2, 0, 1, 0, 0, 2, 2, 1, 1, 2, 2, 1, 2, 0, 0,
       1, 0, 2, 2, 0, 2, 0, 1, 2, 1, 1, 2, 2, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 2, 2, 1, 2, 1, 0, 1, 1, 0, 2, 1, 0, 0, 1, 0, 2, 2, 1, 1, 2,
       1, 2, 1, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 1, 0, 1, 0], dtype=int64)

In [23]:
labels

array([2., 1., 0., 0., 0., 2., 1., 0., 0., 0., 2., 0., 2., 0., 0., 2., 0.,
       0., 2., 2., 0., 0., 0., 2., 0., 0., 1., 0., 2., 0., 2., 2., 2., 1.,
       1., 2., 2., 0., 1., 1., 1., 1., 2., 1., 2., 2., 1., 0., 2., 0., 2.,
       1., 1., 1., 2., 2., 1., 1., 0., 0., 1., 0., 1., 1., 2., 1., 1., 2.,
       2., 0., 1., 1., 0., 2., 0., 1., 0., 0., 2., 2., 1., 1., 2., 2., 1.,
       2., 0., 0., 1., 0., 2., 2., 0., 2., 0., 1., 2., 1., 1., 2., 2., 0.,
       1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 2., 2., 1., 2., 1., 0., 1.,
       1., 0., 2., 1., 0., 0., 1., 0., 2., 2., 1., 1., 1., 1., 1., 1., 2.,
       2., 0., 2., 2., 0., 2., 2., 2., 2., 0., 1., 0., 1., 0.])